In [21]:
import pysal
import geopandas as gpd
import arcpy
from pysal.model import mgwr
import pandas as pd
import os

In [28]:
cwd = os.getcwd()
arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

In [7]:
stations_df = pd.read_csv('stations_5-2.csv')
pm_df = pd.read_csv('historicAirQuality_4-30.csv')
merged_df = pd.merge(stations_df, pm_df, on='sensor_index')

In [15]:
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])

# Filter for readings on May 2
april_2_df = merged_df.loc[merged_df['timestamp'].dt.date == pd.Timestamp('2023-04-02').date()]

In [32]:
weekday_avgs = april_2_df.groupby('sensor_index').mean().reset_index()
weekday_avgs.head()
weekday_avgs.to_csv('weekday_avgs.csv', index=False)
avgs_path = os.path.join(cwd, '..', '..', 'data', 'weekday_avgs.csv')

In [74]:
merged_df

,sensor_index,last_modified,date_created,last_seen,name,firmware_version,uptime,position_rating,latitude,longitude,altitude,channel_state,channel_flags,timestamp,humidity,temperature,pressure,pm2_5
0,3088,2017-09-09 21:42:29,2017-08-29 21:03:53,2023-05-02 19:42:13,Howe Neighborhood,6.06b,8871,5,44.935818,-93.21752,833,3,0,2022-09-04,43.297,74.176,993.320,0.4255
1,3088,2017-09-09 21:42:29,2017-08-29 21:03:53,2023-05-02 19:42:13,Howe Neighborhood,6.06b,8871,5,44.935818,-93.21752,833,3,0,2023-03-31,61.289,43.874,972.402,6.7030
2,3088,2017-09-09 21:42:29,2017-08-29 21:03:53,2023-05-02 19:42:13,Howe Neighborhood,6.06b,8871,5,44.935818,-93.21752,833,3,0,2023-04-12,30.396,81.503,976.471,5.7295
3,3088,2017-09-09 21:42:29,2017-08-29 21:03:53,2023-05-02 19:42:13,Howe Neighborhood,6.06b,8871,5,44.935818,-93.21752,833,3,0,2022-12-14,62.533,43.450,976.015,2.9040
4,3088,2017-09-09 21:42:29,2017-08-29 21:03:53,2023-05-02 19:42:13,Howe Neighborhood,6.06b,8871,5,44.935818,-93.21752,833,3,0,2023-01-29,34.392,14.193,999.315,0.6595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16665,128195,2022-02-10 19:55:03,2021-09-16 22:40:18,2023-05-02 19:42:15,Groveland Park,7.02,18296,5,44.935780,-93.18634,926,3,0,2022-10-11,36.957,70.990,976.219,7.3895
16666,128195,2022-02-10 19:55:03,2021-09-16 22:40:18,2023-05-02 19:42:15,Groveland Park,7.02,18296,5,44.935780,-93.18634,926,3,0,2022-11-01,32.001,61.188,981.327,10.2005
16667,128195,2022-02-10 19:55:03,2021-09-16 22:40:18,2023-05-02 19:42:15,Groveland Park,7.02,18296,5,44.935780,-93.18634,926,3,0,2022-07-10,47.633,84.253,983.682,4.9420
16668,128195,2022-02-10 19:55:03,2021-09-16 22:40:18,2023-05-02 19:42:15,Groveland Park,7.02,18296,5,44.935780,-93.18634,926,3,0,2023-03-27,33.946,40.133,990.732,2.1750


In [35]:
weekday_avgs.head()

,sensor_index,uptime,position_rating,latitude,longitude,altitude,channel_state,channel_flags,humidity,temperature,pressure,pm2_5
0,3088,8871.0,5.0,44.935818,-93.217520,833.0,3.0,0.0,44.785,47.498,979.076,3.9890
1,11134,57395.0,5.0,44.927760,-93.322350,886.0,3.0,1.0,0.000,32.000,0.000,7.9375
2,21179,46473.0,5.0,44.965965,-93.284615,891.0,3.0,0.0,54.361,44.135,973.346,9.1245
3,23989,32519.0,5.0,45.112064,-93.277510,863.0,3.0,0.0,54.479,41.036,979.271,9.5675
4,43305,46697.0,5.0,44.899097,-93.301690,904.0,3.0,0.0,48.011,43.903,977.818,10.4185


In [34]:
output_point_fc = "weekday_avgs_XY"
output_point_path = os.path.join(save_path_gdb, output_point_fc)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
station_points = arcpy.management.XYTableToPoint(avgs_path, output_point_path, x_field, y_field)

In [70]:
x_names = [ # Independents
    'humidity', 
    'temperature',
    'pressure'
    ]

explanatory_vars = ';'.join(x_names)

arcpy.stats.GeographicallyWeightedRegression('weekday_avgs_XY', 'pm2_5', explanatory_vars, 'week_gwr', "FIXED", 'BANDWIDTH PARAMETER')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output feature class: Dataset C:\Users\tande\Documents\GitHub\QualityAirQualityCities\arcpy\Modeling\..\..\data\QAQC.gdb\week_gwr already exists.
Failed to execute (GeographicallyWeightedRegression).


In [75]:
import numpy as np

no_nans = merged_df.dropna().copy()
coords = np.array(list(zip(no_nans.longitude, no_nans.latitude)))
y = no_nans.pm2_5.values.reshape(-1, 1)
X = no_nans[x_names].values
# s.reshape(-1, 1)
bw = 0.1

In [76]:
y.ndim

2

In [77]:
m1 = mgwr.gwr.GWR(coords = coords,
y = y,
X = X,
fixed = True,
bw = bw)


results = m1.fit()

results.summary()

Model type                                                         Gaussian
Number of observations:                                               16146
Number of covariates:                                                     4

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       1396148809.621
Log-likelihood:                                                 -114680.460
AIC:                                                             229368.919
AICc:                                                            229370.923
BIC:                                                           1395992402.880
R2:                                                                   0.049
Adj. R2:                                                              0.049

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- --